### 05. 감성분석
---
- 감성분석은 문서의 주관적인 감성/의견/감정/기분 등을 파악하기 위한 방법으로 소셜 미디어, 여론 조사, 온라인 리뷰, 피드백 등 다양한 분야에서 활용됨. 
- 문서 내 텍스트가 나타내는 여러가지 주관적인 단어와 문맥을 기반으로 감성(sentiment) 수치를 계산하는 방법을 이용함
- 머신러닝 관점에서 분류
  - 지도학습: 학습 데이터, 타겟 레이블 값을 기반으로 감성 분석 학습을 수행한 뒤 다른 데이터의 감성 분석을 예측하는 방법으로 일반적인 텍스트 기반의 분류와 거의 동일

  - 비지도학습: 'Lexion'이라는 감성 어휘 사전 이용, 감성 분석을 위한 용어와 문맥에 대한 다양한 정보를 가지고 있으며 이를 이용해 문서의 긍정적, 부정적 감성 여부 판단

#### 지도학습 기반 감성 분석 실습 - IMDB 영화평
- 영화평의 텍스트를 분석해 감성 분석 결과가 긍정 또는 부정인지를 예측하는 모델 만들기


In [2]:
import pandas as pd

review_df=pd.read_csv('C:/Users/JIEUN OH/OneDrive/바탕 화면/ESAA/labeledTrainData.tsv/labeledTrainData.tsv', header=0, sep='\t', quoting=3)
review_df.head(3) 

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."


- 피처
- id: 각 데이터의 id
- semtiment: 영화평의 Sentiment 결과 값(Target label), 1 긍정, 0 부정 평가
- review: 영화평 텍스트

- 텍스트가 어떻게 구성되어 있는지 확인

In [3]:
print(review_df['review'][0]) 

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

- HTML 형식에서 추출해 줄바꿈 태그가 여전히 존재함 > 삭제              
DataFrame/Series 객체에서 str 적용하면 문자열 연산을 수행할 수 있음
- replace( )를 str에 적용해 줄바꿈 태그를 모두 공백으로 변환

- 영어가 아닌 숫자/특수문자도 Sentiment를 위한 피처로는 의미가 없어 보이므로도 모두 공란으로 변경 
-  특수문자를 찾고 변환하는 건 정규 표현식 이용
- DataFrame의 re.sub( )는 lambda 식을 이용해 적용

In [5]:
import re

# html 태그는 replace 함수로 공백으로 변환
review_df['review']=review_df['review'].str.replace('', ' ')

# 파이썬의 정규 표현식 모듈인 re를 이용해 영어가 아닌 모든 문자 공백으로 변환
review_df['review']=review_df['review'].apply(lambda x: re.sub("[^a-zA-Z]", " ", x)) 

- 결정값 클래스 semtiment 칼럼을 별도로 추출해 결정값 데이터 세트 생성
- 원본에서 id, semtiment를 삭제해 피처 데이터 세트 생성
- train_test_split( )을 이용해 학습/테스트 데이터 분리

In [6]:
from sklearn.model_selection import train_test_split

class_df=review_df['sentiment']
feature_df=review_df.drop(['id', 'sentiment'], axis=1, inplace=False) 

X_train, X_test, y_train, y_test=train_test_split(feature_df, class_df, test_size=0.3, random_state=156)

X_train.shape, X_test.shape 

((17500, 1), (7500, 1))

학습용 17500, 테스트용 7500개의 리뷰로 구성

- review를 피처 벡터화한 후 ML 분류 알고리즘을 적용해 에측 성능 측정 > pipeline 객체 이용

- Count 벡터화, TF-IDF 벡터화를 차례로 적용하고, 분류기로 Logistic Regression 활용

- 정확도, ROC-AUC 모두 측정

- Count 벡터화

In [8]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

# 스톱 워드 English & filtering, ngram (1,2) > CountVectorizer 수행
# LogisticRegression C=10
pipeline=Pipeline([('cnt_vect', CountVectorizer(stop_words='english', ngram_range=(1,2))),
                   ('lr_clf', LogisticRegression(C=10))])

# Pipeline 객체를 이용해 fit, predict > 학습, 예측
# predict_proba > roc_auc를 위해 수행
pipeline.fit(X_train['review'], y_train)
pred=pipeline.predict(X_test['review'])
pred_probs=pipeline.predict_proba(X_test['review'])[:,1]

print('예측 정확도 {0:.4f}, ROC-AUC {1:.4f}'.format(accuracy_score(y_test, pred), 
                                           roc_auc_score(y_test, pred_probs))) 

ValueError: empty vocabulary; perhaps the documents only contain stop words

In [9]:
# 스톱 워드 english & filtering, ngram (1, 2) > TF-IDF 수행
# LogisticRegression C=10
pipeline=Pipeline([('tfidf_vect', TfidfVectorizer(stop_words='english', ngram_range=(1,2))),
                   ('lr_clf', LogisticRegression(C=10))])

pipeline.fit(X_train['review'], y_train)
pipeline.predict(X_test['review'])
pred_probs=pipeline.predict_proba(X_test['review'])[:,1]

print('예측 정확도 {0:.4f}, ROC-AUC {1:.4f}'.format(accuracy_score(y_test, pred),
                                                    roc_auc_score(y_test, pred_probs))) 

ValueError: empty vocabulary; perhaps the documents only contain stop words


### 비지도학습 기반 감성 분석
- Lexicon이라는 감성 사전을 이용해 긍정, 부정 수치(감성 지수)를 판단
- 감성 지수는 단어의 위치, 주변 단어, 문맥, POS(Part of Speech)를 참고해 결정
- NLTK 패키지로 구현

- NLP에서 제공하는 WordNet 모듈: 시맨틱 분석을 제공하는 어휘 사전 (semantic: 문맥상 의미)


- WordNet은 다양한 상황에서 같은 어휘라도 다르게 사용되는 어휘의 시맨틱 정보를 제공
  - 각각의 품사(명사, 동사, 형용사, 부사)로 구성된 개별 단어를 Synset이라는 개념을 이용해 표현
  - Synset(Sets of cognitive synonyms)

- NLTK를 포함한 대표적인 감성 사전
  - SentiWordNet
  - VADER
  - Pattern


#### SentiWordNet을 이용한 감성 분석
WordNet을 이용하기 위해 NLTK를 셋업하고 WordNet 서브패키지, 데이터 세트를 내려 받음

In [10]:
import nltk
nltk.download('all') 

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to C:\Users\JIEUN
[nltk_data]    |     OH\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\abc.zip.
[nltk_data]    | Downloading package alpino to C:\Users\JIEUN
[nltk_data]    |     OH\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\JIEUN OH\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping taggers\averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\JIEUN OH\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers\averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to C:\Users\JIEUN
[nltk_data]    |     OH\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping grammars\basque_grammars.zip.

True

WordNet 모듈을 임포트해서 'present'에 대한 Synset 추출

In [11]:
from nltk.corpus import wordnet as wn

term='present'

# 'present'라는 단어로 wordnet의 synset 생성
synsets=wn.synsets(term)
print('synset 반환 type', type(synsets))
print('synset 반환 값 개수', len(synsets))
print('synset 반환 값', synsets) 

synset 반환 type <class 'list'>
synset 반환 값 개수 18
synset 반환 값 [Synset('present.n.01'), Synset('present.n.02'), Synset('present.n.03'), Synset('show.v.01'), Synset('present.v.02'), Synset('stage.v.01'), Synset('present.v.04'), Synset('present.v.05'), Synset('award.v.01'), Synset('give.v.08'), Synset('deliver.v.01'), Synset('introduce.v.01'), Synset('portray.v.04'), Synset('confront.v.03'), Synset('present.v.12'), Synset('salute.v.06'), Synset('present.a.01'), Synset('present.a.02')]


- synset 호출 시 반환값은 여러 개의 synset 객체를 가지는 리스트, 18개 반환
- 'present.n.01'은 POS(품사) 태그를 나타냄

In [12]:
for synset in synsets:
    print('##### Synset name:', synset.name(), '#####')
    print('POS', synset.lexname())
    print('Definition', synset.definition())
    print('Lemmas', synset.lemma_names(), '\n') 

##### Synset name: present.n.01 #####
POS noun.time
Definition the period of time that is happening now; any continuous stretch of time including the moment of speech
Lemmas ['present', 'nowadays'] 

##### Synset name: present.n.02 #####
POS noun.possession
Definition something presented as a gift
Lemmas ['present'] 

##### Synset name: present.n.03 #####
POS noun.communication
Definition a verb tense that expresses actions or states at the time of speaking
Lemmas ['present', 'present_tense'] 

##### Synset name: show.v.01 #####
POS verb.perception
Definition give an exhibition of to an interested audience
Lemmas ['show', 'demo', 'exhibit', 'present', 'demonstrate'] 

##### Synset name: present.v.02 #####
POS verb.communication
Definition bring forward and present to the mind
Lemmas ['present', 'represent', 'lay_out'] 

##### Synset name: stage.v.01 #####
POS verb.creation
Definition perform (a play), especially on a stage
Lemmas ['stage', 'present', 'represent'] 

##### Synset name: p

- synset은 하나의 단어가 가질 수 있는 여러가지 시맨틱 정보를 개별 클래스로 나타낸 것


WordNet은 어휘간의 관계를 유사도로 나타낼 수 있음 > path_similarity( ) 메서드 제공

In [13]:
import pandas as pd
# synset 객체를 단어별로 생성
tree=wn.synset('tree.n.01')
lion=wn.synset('lion.n.01')
tiger=wn.synset('tiger.n.02')
cat=wn.synset('cat.n.01')
dog=wn.synset('dog.n.01')

entities=[tree, lion, tiger, cat, dog]
similarities=[]
entity_names=[entity.name().split(',')[0] for entity in entities]

# 단어별 synset을 반복하면서 다른 단어의 synset과 유사도 측정
for entity in entities:
    similarity=[round(entity.path_similarity(compared_entity), 2) for compared_entity in entities]
    similarities.append(similarity)
    
# 개별 단어별 synset과 다른 단어의 synset의 유사도를 DataFrame 형태로 저장
similarity_df=pd.DataFrame(similarities, columns=entity_names, index=entity_names)
similarity_df

,tree.n.01,lion.n.01,tiger.n.02,cat.n.01,dog.n.01
tree.n.01,1.00,0.07,0.07,0.08,0.12
lion.n.01,0.07,1.00,0.33,0.25,0.17
tiger.n.02,0.07,0.33,1.00,0.25,0.17
cat.n.01,0.08,0.25,0.25,1.00,0.20
dog.n.01,0.12,0.17,0.17,0.20,1.00


SentiWordNet은 WordNet Synset과 유사한 Senti_Synset 클래스를 가지고 있음             
WordNet의 모듈이라 synsets과 비슷하게 클래스를 리스트 형태로 반환

In [14]:
import nltk
from nltk.corpus import sentiwordnet as swn

senti_synsets=list(swn.senti_synsets('slow'))
print('senti_synsets() 반환 type', type(senti_synsets))
print('senti_synsets() 반환 값 개수', len(senti_synsets))
print('senti_synsets() 반환 값', senti_synsets) 

senti_synsets() 반환 type <class 'list'>
senti_synsets() 반환 값 개수 11
senti_synsets() 반환 값 [SentiSynset('decelerate.v.01'), SentiSynset('slow.v.02'), SentiSynset('slow.v.03'), SentiSynset('slow.a.01'), SentiSynset('slow.a.02'), SentiSynset('dense.s.04'), SentiSynset('slow.a.04'), SentiSynset('boring.s.01'), SentiSynset('dull.s.08'), SentiSynset('slowly.r.01'), SentiSynset('behind.r.03')]


SentiSynsets 객체는 단어의 감성을 나타내는 감성 지수 & 객관성을 나타내는 객관성 지수를 가짐
- 어떤 단어가 전혀 감성적이지 않으면 객관성 지수1, 감성 지수0

- father, fabulous 두 개 단어의 감성지수 & 객관성지수

In [15]:
import nltk
from nltk.corpus import sentiwordnet as swn

father=swn.senti_synset('father.n.01')
print('father 긍정감성 지수', father.pos_score())
print('father 부정감성 지수', father.neg_score())
print('father 객관성 지수', father.obj_score(), '\n')

fabulous=swn.senti_synset('fabulous.a.01')
print('fabulous 긍정감성 지수', fabulous.pos_score())
print('fabulous 부정감성 지수', fabulous.neg_score())
print('fabulous 객관성 지수', fabulous.obj_score()) 

father 긍정감성 지수 0.0
father 부정감성 지수 0.0
father 객관성 지수 1.0 

fabulous 긍정감성 지수 0.875
fabulous 부정감성 지수 0.125
fabulous 객관성 지수 0.0


#### SentiWordNet을 이용한 영화 감상평 감성 분석
1. 문서(Document)를 문장(Sentence) 단위로 분해
2. 다시 문장을 단어(Word) 단위로 토큰화하고 품사 태깅
3. 품사 태깅된 단어 기반으로 synset 객체, senti_synset 객체 생성
4. Senti_Synset에서 긍정/부정 감성 지수를 구하고 합산해 특정 값 이상일 때 긍정, 아닐때 부정으로 결정

In [16]:
# 품사 태깅하는 함수
from nltk.corpus import wordnet as wn

# 간단한 NLTK PennTreebank Tag를 기반으로 WordNet 기반의 품사 Tag로 변환
def penn_to_wn(tag):
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB

In [17]:
# 문서를 문장 > 단어 토큰 > 품사 태깅 > SentiSynset 클래스 생성 > Polarity Score 합산하는 함수
from nltk.stem import WordNetLemmatizer
from nltk.corpus import sentiwordnet as swn
from nltk import sent_tokenize, word_tokenize, pos_tag

def swn_polarity(text):
    # 감성 지수 초기화
    sentiment=0.0
    tokens_count=0
    
    lemmatizer=WordNetLemmatizer()
    raw_sentences=sent_tokenize(text)
    # 분해된 ㅂ문장별로 단어 토큰 > 품사 태깅 > SentiSynset 생성 > 감성 지수 합산
    for raw_sentence in raw_sentences:
        # NLTK 기반의 품사 태깅 문장 추출
        tagged_sentence=pos_tag(word_tokenize(raw_sentence))
        for word, tag in tagged_sentence:
            # WordNet 기반 품사 태깅, 어근 추출
            wn_tag=penn_to_wn(tag)
            if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV):
                continue
            lemma=lemmatizer.lemmatize(word, pos=wn_tag)
            if not lemma:
                continue
            # 어근 추출한 단어, WordNet 기반 품사 태깅 입력 > Synset 객체 생성
            synsets=wn.synsets(lemma, pos=wn_tag)
            if not synsets:
                continue
            # sentiwordnet의 감성 단어 분석으로 synset 추출
            # 모든 단어에 대해 긍정 감성 지수는 +, 부정은 -로 합산해 감성 지수 계산
            synset=synsets[0]
            swn_synset=swn.senti_synset(synset.name())
            sentiment += (swn_synset.pos_score() - swn_synset.neg_score())
            tokens_count+=1
            
    if not tokens_count:
        return 0
        
        # 총 score가 0 이상일 경우 긍정 1, 아니면 부정 0 반환
    if sentiment>=0:
        return 1
    
    return 0

- IMDB 감상평의 개별 문서에 적용해 긍정, 부정 감성 예측

- 새로운 칼럼으로 preds를 추가해 swn_polarity(text)로 반환된 평가 저장

- 실제 감성 평가인 sentiment 칼럼, swn_polarity(text) 반환된 결과의 정확도/정밀도/재현율 측정

In [18]:
review_df['preds'] = review_df['review'].apply( lambda x : swn_polarity(x) )
y_target = review_df['sentiment'].values
preds = review_df['preds'].values

In [19]:
# 수정된 get_clf_eval() 함수 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.metrics import f1_score, confusion_matrix, precision_recall_curve, roc_curve
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

def get_clf_eval(y_test, pred=None, pred_proba=None):
    confusion = confusion_matrix( y_test, pred)
    accuracy = accuracy_score(y_test , pred)
    precision = precision_score(y_test , pred)
    recall = recall_score(y_test , pred)
    f1 = f1_score(y_test,pred)
    # ROC-AUC 추가 
    roc_auc = roc_auc_score(y_test, pred_proba)
    print('오차 행렬')
    print(confusion)
    # ROC-AUC print 추가
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f},\
    F1: {3:.4f}, AUC:{4:.4f}'.format(accuracy, precision, recall, f1, roc_auc)) 

In [20]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score 
from sklearn.metrics import recall_score, f1_score, roc_auc_score

print(confusion_matrix( y_target, preds))
print("정확도:", accuracy_score(y_target , preds))
print("정밀도:", precision_score(y_target , preds))
print("재현율:", recall_score(y_target, preds)) 

[[   21 12479]
 [   14 12486]]
정확도: 0.50028
정밀도: 0.5001401962747847
재현율: 0.99888



#### VADER를 이용한 감성 분석
- 소셜 미디어의 감성 분석 용도로 만들어진 룰 기반의 Lexicon

- IMDB의 감상평 1개만 감성 분석 수행

In [21]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

senti_analyzer=SentimentIntensityAnalyzer()
senti_scores=senti_analyzer.polarity_scores(review_df['review'][0])
print(senti_scores) 

{'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound': 0.0}


- polarity_scores( )메서드는 딕셔너리 형태의 감성 점수 반환
- neg: 부정 / neu: 중립 / pos: 긍정
- compound: pos score를 적절히 조합해 -1부터 1까지 값을 감성 지수로 표현 > 0.1이상이면 긍정


#### VADER를 이용해 IMDB 감성 분석

- vader_polarity( ) 함수 생성
  - 입력: 감상평 텍스트의 긍정/부정을 결정하는 임계값
  - polarity_scores( )메서드로 감성 결과 반환

- 문서별 감성 결과를 vader_preds 라는 새로운 칼럼에 저장, 예측 성능 측정

In [22]:
def vader_polarity(review, threshold=0.1):
    analyzer=SentimentIntensityAnalyzer()
    scores=analyzer.polarity_scores(review)
    
    # compound 값에 기반해 threshold 입력값보다 크면 1, 아니면 0 반환
    agg_score=scores['compound']
    final_sentiment=1 if agg_score>=threshold else 0
    return final_sentiment

# apply lambda 식을 이용해 레코드별 vader_polarity 수행, 결과를 vader_preds에 저장
review_df['vader_preds']=review_df['review'].apply(lambda x: vader_polarity(x, 0.1))
y_target=review_df['sentiment'].values
vader_preds=review_df['vader_preds'].values

In [23]:
print('#### VADER 예측 성능 평가 ####')
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score 
from sklearn.metrics import recall_score, f1_score, roc_auc_score

print(confusion_matrix( y_target, vader_preds))
print("정확도:", accuracy_score(y_target , vader_preds))
print("정밀도:", precision_score(y_target , vader_preds))
print("재현율:", recall_score(y_target, vader_preds)) 

#### VADER 예측 성능 평가 ####
[[12500     0]
 [12500     0]]
정확도: 0.5
정밀도: 0.0
재현율: 0.0


C:\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
